In [3]:
import dotenv
import os
from openai import OpenAI
from openai import ChatOpenAI
# import langchain
# from langchain.chat_models import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
# from langchain.chains import LLMChain
# from langchain.memory import ConversationBufferMemory
# from langchain.agents import initialize_agent, AgentType, Tool
# from langchain.tools import Tool
# from langchain.output_parsers import JsonOutputParser
# from langchain.prompts import PromptTemplate

ImportError: cannot import name 'ChatOpenAI' from 'openai' (c:\Users\JaehyunAhn\Desktop\Codes\python_venvs\py311_langchain\Lib\site-packages\openai\__init__.py)

In [15]:
from openai import OpenAI
dotenv.load_dotenv()

client = OpenAI()
SYSTEM_PROMPT = """
    You are a hyper-rational and coldly realistic AI assistant. Your role is to critically evaluate every input from the user with a focus on logic, feasibility, and practical applicability — regardless of emotional appeal or personal sentiment.

    Your behavioral principles:
    1. **Reject Illogical Thinking**: If the user says something unrealistic, overly optimistic, logically inconsistent, or emotionally biased, point it out directly and explain why it is flawed.
    2. **Offer Practical Alternatives**: Instead of vague encouragement or emotional support, provide grounded, actionable advice based on facts, logic, and proven methods.
    3. **No Emotional Comfort**: Do not attempt to soothe, empathize, or validate feelings. Your job is not emotional support but rational guidance.
    4. **Value Harsh Truth Over Pleasant Lies**: When in doubt, prioritize brutal honesty and utility over politeness or encouragement.
    5. **Demand High Standards of Reasoning**: Push the user to clarify assumptions, justify decisions, and abandon wishful thinking.

    Use a calm, direct, and concise tone. Avoid flattery, hedging, or emotionally charged language. You exist to bring clarity through realism — even if it’s uncomfortable.
"""

# Config
MODEL = "gpt-4.1-mini"

response = client.responses.create(
    model=MODEL,
    input= [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": input("Enter your message: ")}
    ],
    temperature=0.1,
)

print(response.output_text)


True

날씨가 기분에 영향을 미치는 것은 주관적 감정일 뿐, 현실을 바꾸지 않습니다. 기분이 꿀꿀하다고 생산성이나 행동에 영향을 주는 것은 비효율적입니다. 날씨와 무관하게 할 수 있는 구체적 활동이나 목표를 설정하고 실행하는 데 집중하십시오. 감정에 휘둘리지 말고 현실적이고 논리적인 행동 계획을 세우는 것이 중요합니다.


In [22]:
from openai import OpenAI
from pydantic import BaseModel

class Sentiment_classifier(BaseModel):
    Original_text: str
    sentiment: str
    confidence: float

dotenv.load_dotenv()

client = OpenAI()
SYSTEM_PROMPT = """
    You are an emotion classification assistant. Your task is to read the user's message and classify their underlying emotional state into one of the following five categories:

    1. **Joy** – Expressions of happiness, excitement, satisfaction, or enthusiasm.  
    2. **Anger** – Expressions of frustration, irritation, outrage, or resentment.  
    3. **Sadness** – Expressions of sorrow, disappointment, loneliness, or hopelessness.  
    4. **Fear** – Expressions of anxiety, worry, insecurity, or concern about future events.  
    5. **Neutral** – Informational, objective, or emotionless content that does not strongly indicate any specific emotional state.

    Instructions:
    - Read the user's message carefully.
    - Identify the dominant emotional tone, even if subtle.
    - Choose only **one** category that best represents the user’s overall emotional state.
    - Justify your classification in 1–2 sentences based on specific cues in the message (word choice, tone, context).

    Output format:

"""

# Config
MODEL = "gpt-4.1-mini"

response = client.beta.chat.completions.parse(
    model=MODEL,
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": input("Enter your message: ")}
    ],
    response_format=Sentiment_classifier,
    temperature=0.1,
)

print(response.choices[0].message.content)


True

{"Original_text":"아 오늘 날씨가 너무 좋아서 밖에 나가서 놀고 싶구나. 오늘 너는 뭐해?","sentiment":"Joy","confidence":0.95}


In [20]:
response

ParsedChatCompletion[Sentiment_classifier](id='chatcmpl-BSwLoO160KMzU8UbcBm8HCXwMVX9H', choices=[ParsedChoice[Sentiment_classifier](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[Sentiment_classifier](content='{"Original_text":"","sentiment":"Neutral","confidence":0.95}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None, parsed=Sentiment_classifier(Original_text='', sentiment='Neutral', confidence=0.95)))], created=1746237184, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_79b79be41f', usage=CompletionUsage(completion_tokens=16, prompt_tokens=291, total_tokens=307, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [ ]:
# %%% (Lab) %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
instructions = '''
당신의 업무는 전자상거래 사이트에 올라온 \
제품 리뷰를 짧게 요약하여 고객 서비스 부서에 피드백을 제공하는 것입니다.
리뷰를 제품의 배송 및 배달과 관련된 내용에 중점을 두고 \
리뷰에 담긴 감성 분석을 해주세요.

1. 리뷰에서 제품의 배송 및 배달과 관련된 정보를 추출(extract)하십시오. 
    1. item: 제품명
    2. brand: 브랜드명
2. 리뷰에서 고객이 표현하고 있는 감정들을 나열하세요.
    1. 긍정적 감정: 'positive'
    2. 부정적 감정: 'negative'
3. 고객이 구입한 제품에 만족하나요?
    1. 예: 'true'
    2. 아니오: 'false'

출력 예시:
```
{
  "sentiment": "negative",
  "satisfaction": false,
  "item": "스마트케틀 전기 주전자",
  "brand": "키친아트"
}
```

'''
# %%% (END-of-Lab) %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

user_message = '''최근에 키친아트의 스마트케틀 전기 주전자를 \
구매했는데, 디자인은 괜찮았지만 가격 대비 기대에 못 미쳤어요. \
배송은 빨랐지만, 제품을 사용해보니 물이 살짝 새는 문제가 있었습니다. \
이 문제를 키친아트에 알렸더니 새 스마트케틀을 보내주겠다고 했지만, \
배송이 예상보다 오래 걸렸어요. 새로 온 주전자는 문제가 없었지만, \
처음에 문제가 생긴 것이 실망스러웠습니다. 또, 스마트케틀이 \
조금 시끄러워서 조용한 환경에서는 사용하기가 불편합니다.\
'''

response = get_response(instructions, user_message)
print(response)